In [2]:
import pandas as pd
from collections import Counter

In [11]:
df = pd.read_csv('data/merged_data.csv')

In [ ]:
df.shape

In [5]:
def get_state(answer, guess):
    count_letters = Counter(answer)
    state = [0] * 5
    # print(count_letters)

    # This marks the greens
    for index, letter in enumerate(guess):
        # print(letter)
        if letter in count_letters:
            # print(f'letter: {letter}')
            if letter == answer[index] and count_letters[letter] > 0:
                state[index] = 2
                count_letters[letter] -= 1

    # This marks the yellows
    for index, letter in enumerate(guess):
        if letter in count_letters:
            if count_letters[letter] > 0 and state[index] == 0:
                state[index] = 1
                count_letters[letter] -= 1

    subset = [0] * 3

    for s in state:
        subset[2-s] += 1

    return "".join(map(str, subset))

In [ ]:
answer = 'banal'
guess = 'daddy'

print(get_state(answer, guess))

In [ ]:
for index, row in df.iterrows():
    # drop the row if the guess is not 5 letters
    if len(eval(row['wordle_guesses'])) != len(eval(row['optimal'])) + 1:
        df.drop(index, inplace=True)

def check_if_guess_duplicate_as_answer(df):
    for idx, row in df.iterrows():
        if row['wordle_guesses'].count(row['wordle_answer']) != 1:
            # delete current row
            # print(row['entry_id'])
            df.drop(idx, inplace=True)

check_if_guess_duplicate_as_answer(df)


In [ ]:
df.shape

In [ ]:
# make a new data frame called state_data where we only keep the optimal and answer columns
data_needed = df[['optimal', 'wordle_answer', 'wordle_guesses']].copy()
data_needed.head()

In [50]:
# make a test dataframe that is the first 10 rows of the data_needed dataframe
test_data = data_needed.head(10)

In [ ]:
# specify the data types
# still some issues with "['later', 'humor', 'salet', 'spill']",4,humor
from tqdm import tqdm
state_data_final = pd.DataFrame(columns=['state', 'previous_guesses', 'optimal_guess', 'human_guess'])

previous_dict = {}
optimal_dict = {}
human_dict = {}

# iterate over the rows of the data frame
for index, row in tqdm(data_needed.iterrows(), total=data_needed.shape[0]):
    answer = row['wordle_answer']
    guesses = eval(row['wordle_guesses'])
    optimal = eval(row['optimal'])

    for i in range(len(guesses) - 1):
        state = get_state(answer, guesses[i])
        try:
            opt_guess = optimal[i][1][0]
        except:
            print(index)
            print(optimal)
            print(i)
            print(guesses)
        # print(opt_guess)
        # check if the state is already in the state column
        if state not in previous_dict:
            previous_dict[state] = []
            optimal_dict[state] = []
            human_dict[state] = []

        previous_dict[state].append(guesses[:i+1])
        optimal_dict[state].append(opt_guess)
        human_dict[state].append(guesses[i+1])
        
for state in previous_dict:
    state_data_final = state_data_final.append({'state': state, 'previous_guesses': previous_dict[state], 'optimal_guess': optimal_dict[state], 'human_guess': human_dict[state]}, ignore_index=True)

# state_data_final.to_csv('state_data.csv', index=False)
        





In [16]:
state_data_final.to_csv('data/state_data.csv', index=False)